In [1]:
import json
import glob
import os
import datetime
import re
import pandas as pd
from slack_data_loader import SlackLoader


In [2]:
data_folder = '/Users/alex/Documents/ODS/opendatascience Slack export May 20 2017/'
ods = SlackLoader(data_folder,only_channels=('welcome',))

In [3]:
df_msg = pd.DataFrame.from_records(ods.messages)
df_msg.fillna(0).groupby(['type'])['dt'].count()

type
message    3992
Name: dt, dtype: int64

In [6]:
print(df_msg.text[4])

как пример хорошего :smiley:


Вытащим сообщения-представления

In [2]:
df_msg.head()

In [8]:
k = 0
hi_messages = []
hi_tokens=('все', 'привет','добр','шалом','салют','здрав','хай','я','ребят','коллег')
for _, i in df_msg.iterrows():
    if i.text.lower().startswith(hi_tokens) and len(i.text) > 100:
        k+=1
        hi_messages.append(i.text)

In [1]:
for i in hi_messages[-10:]:
    print(i)
    print('='*80)

In [11]:
from pymystem3 import Mystem
mystem = Mystem()
# Installing mystem to /home/dmchk/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
# Экземпляр класса Mystem предоставляет метод lemmatize, вызывающий mystem с соответствующими параметрами.

Installing mystem to /Users/alex/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-macosx10.8.tar.gz


In [12]:
text = 'Как насчёт небольшого стемминга'
lemmas = mystem.lemmatize(text)
print(''.join(lemmas))

как насчет небольшой стемминг



In [16]:
from random import  choice

In [32]:
import matplotlib.pyplot as plt

In [68]:
from IPython.display import HTML

In [37]:
import numpy as np

In [75]:
def get_n_colors_on_colormap(n):
    cmap = plt.cm.get_cmap('jet')
    # cmap = matplotlib.cm.get_cmap('Spectral')
    rg = np.linspace(0.3,1.0,n)
    cols = np.asarray(np.floor(255*cmap(rg)),dtype=int)
    get_color_hash = lambda x: '#%02x%02x%02x' % tuple(x.tolist())
    return list(map(get_color_hash, cols[:,:3]))

In [87]:
tags = ('S','SPRO','V','ADVPRO','A','PR','ADV','APRO','CONJ', 'NUM', 'ANUM', 'PART')
tags_to_color_mapping = dict(zip(tags, get_n_colors_on_colormap(len(tags))))

In [92]:
def plot_seq_with_tags(seq):
    tmplt = '<p>{}</p>'
    span_tmlpt = """<span style="background-color: {color};">{content}</span>"""
    fulltext = ''
    for _word, _pos in seq:
        if _pos is not None:
            fulltext += span_tmlpt.format(color=tags_to_color_mapping[_pos], content=_word)
        else:
            fulltext += _word
    return HTML(tmplt.format(fulltext))

In [79]:
def get_tagging_for_text(rando):
    analysis = mystem.analyze(rando)
    seq = []
    for i in analysis:
        if 'analysis' in i and i['analysis']:
            info = i['analysis'][0]['gr']
            pos = info.split(',')[0]
            if '=' in pos:
                pos = pos.split('=')[0]
            seq.append((i['text'],pos))
        else:
            seq.append((i['text'],None))
    return seq

In [96]:
plot_seq_with_tags(list(zip(tags,tags)))

In [94]:
rando = choice(hi_messages)
seq = get_tagging_for_text(rando)
plot_seq_with_tags(seq)

In [97]:
from operator import itemgetter

In [100]:
Counter(map(itemgetter(1), seq))

Counter({'SPRO': 8,
         None: 290,
         'S': 47,
         'V': 31,
         'ADVPRO': 7,
         'PR': 29,
         'CONJ': 10,
         'PART': 4,
         'A': 15,
         'ANUM': 1,
         'ADV': 5})